In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-intro-to-dl-2021/sample_submission.csv
/kaggle/input/nlp-intro-to-dl-2021/train.csv
/kaggle/input/nlp-intro-to-dl-2021/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Flatten, SpatialDropout1D, Bidirectional,Conv1D, GlobalMaxPooling1D
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
#import torch
#from torchtext import data
#import torch.nn as nn
%matplotlib inline
df_train = pd.read_csv('/kaggle/input/nlp-intro-to-dl-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/nlp-intro-to-dl-2021/test.csv', )
df_train


,text,source
0,Российская сборная лидирует по итогам командно...,mchsgov
1,#СоветМЧС #МЧС #МЧСРОССИИ,mchsgov
2,Инспекторы ГИБДД Москвы приняли участие во Все...,mospolice
3,В Главную Военно-Морскую Базу БФ в г.Балтийск ...,mil
4,Обвиняемые в хищении денежных средств у 32 пож...,mospolice
...,...,...
9876,Расчеты комплексов С-300 выполнили боевые пуск...,mil
9877,#Repost @nataliapetrasheva<br>・・・<br>#щитилира...,mospolice
9878,В столичной полиции наградили победителей реги...,mospolice
9879,ВДВ России получили все бронеавтомобили «Тигр»...,mil


In [3]:
!pip install pymorphy2


     |████████████████████████████████| 55 kB 749 kB/s 
     |████████████████████████████████| 8.2 MB 8.6 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=85648ed1e4a6164701e79faaf12e607cb4cc7e9532d9762acdc2f0fdc603f653
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [4]:
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

In [5]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatize(doc):
    doc = re.sub(patterns, ' ', str(doc))
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    return tokens

In [6]:
df_train['text'] = df_train['text'].apply(lemmatize)

In [7]:
df_test['text'] = df_test['text'].apply(lemmatize)

In [8]:
train_text = df_train['text']
train_cat = df_train['source']
test_text = df_test['text']


In [9]:
max_features = 5000
max_len = 250
tok = Tokenizer(num_words=max_features)
tok.fit_on_texts(list(train_text) + list(test_text))
sequences = tok.texts_to_sequences(train_text)
X_train = sequence.pad_sequences(sequences,maxlen=max_len,padding="post")

test_sequences = tok.texts_to_sequences(test_text)
X_test = sequence.pad_sequences(test_sequences,maxlen=max_len,padding="post")


In [10]:
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)

y_train = to_categorical(y_train, 4)


In [11]:
model = Sequential()
model.add(Embedding(max_features,128, input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
#model.add(Conv1D(128, 7, padding="valid",activation="relu",strides=3))
model.add(Conv1D(64, 7, padding="valid",activation="relu",strides=3))

model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))

model.add(Flatten())

model.add(Dense(4,activation='relu'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 128)          640000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 250, 128)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 82, 64)            57408     
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               197632    
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 1028      
Total params: 896,068
Trainable params: 896,068
Non-trainable params: 0
__________________________________________________

In [12]:
batch_size = 32
model.fit(X_train, y_train, epochs=3 , verbose=1, batch_size = batch_size) 

Epoch 1/3
309/309 [==============================] - 78s 233ms/step - loss: 0.6322 - accuracy: 0.6014
Epoch 2/3
309/309 [==============================] - 72s 233ms/step - loss: 0.1679 - accuracy: 0.9629
Epoch 3/3
309/309 [==============================] - 72s 233ms/step - loss: 0.1282 - accuracy: 0.9756


In [13]:
pred = model.predict(X_test)

In [14]:
print(pred)

[[0.         2.9106402  0.         0.        ]
 [3.8235075  0.         0.         0.        ]
 [0.2696989  0.02917132 0.15604421 0.2705114 ]
 ...
 [3.1203964  0.         0.         0.        ]
 [0.         3.2814345  0.         0.        ]
 [0.         0.         1.1717474  0.        ]]


In [15]:
def f(X):
    mx = 0
    ans = 0
    for i in range(4):
        if X[i] > mx:
            mx = X[i]
            ans = encoder.inverse_transform([i])[0]
    return ans
df_test.text = list(map(f, pred))



In [16]:
df_test.rename(columns={'text':'Category'}, inplace = True)
df_test.index.rename('Id', inplace = True)

In [17]:
df_test.head()

,Category
Id,
0,mil
1,mchsgov
2,russianpost
3,russianpost
4,russianpost


In [18]:
df_test.to_csv("sub.csv")